# EvalML Components

Components are the lowest level of building blocks in EvalML. Each component represents a fundamental operation to be applied to data.

All components accept parameters as keyword arguments to their `__init__` methods. These parameters can be used to configure behavior.

Each component class definition must include a human-readable `name` for the component. Additionally, each component class may expose parameters for AutoML search by defining a `hyperparameter_ranges` attribute containing the parameters in question.

EvalML splits components into two categories: **transformers** and **estimators**.

## Transformers

Transformers subclass the `Transformer` class, and define a `fit` method to learn information from training data and a `transform` method to apply a learned transformation to new data.

For example, an [imputer](../generated/evalml.pipelines.components.SimpleImputer.ipynb) is configured with the desired impute strategy to follow, for instance the mean value. The imputers `fit` method would learn the mean from the training data, and the `transform` method would fill the learned mean value in for any missing values in new data.

All transformers can execute `fit` and `transform` separately or in one step by calling `fit_transform`. Defining a custom `fit_transform` method can facilitate useful performance optimizations in some cases.

In [ ]:
import numpy as np
import pandas as pd
from evalml.pipelines.components import SimpleImputer

X = pd.DataFrame([[1, 2, 3], [1, np.nan, 3]])
display(X)

In [ ]:
imp = SimpleImputer(impute_strategy="mean")
X = imp.fit_transform(X)

display(X)

Below is a list of all transformers included with EvalML:

In [ ]:
from evalml.pipelines.components.utils import all_components, Estimator, Transformer
for component in all_components:
    if issubclass(component, Transformer):
        print(f"Transformer: {component.name}")

## Estimators

Each estimator wraps an ML algorithm. Estimators subclass the `Estimator` class, and define a `fit` method to learn information from training data and a `predict` method for generating predictions from new data. Classification estimators should also define a `predict_proba` method for generating predicted probabilities.

Estimator classes each define a `model_family` attribute indicating what type of model is used.

Here's an example of using the [`LogisticRegressionClassifier`](../generated/evalml.pipelines.components.LogisticRegressionClassifier.rst) estimator to fit and predict on a simple dataset:

In [ ]:
from evalml.pipelines.components import LogisticRegressionClassifier

clf = LogisticRegressionClassifier()

X = X
y = [1, 0]

clf.fit(X, y)
clf.predict(X)

Below is a list of all estimators included with EvalML:

In [ ]:
from evalml.pipelines.components.utils import all_components, Estimator, Transformer
for component in all_components:
    if issubclass(component, Estimator):
        print(f"Estimator: {component.name}")